In [9]:
import pandas as pd

annotator_names = ['Hjalmar', 'Sörme']
annotations = []

for name in annotator_names:
    annotations.append(pd.read_excel('annotations/LLM Prediction Annotations(11).xlsx', sheet_name=name))

for annotation in annotations:
    annotation['Annotation'] = annotation['Annotation LLM 2'].fillna('No description')
    print(annotation.head(3))

             database_name table_name original_column_name  \
0  debit_card_specializing  yearmonth           CustomerID   
1  debit_card_specializing  yearmonth                 Date   
2  debit_card_specializing  yearmonth          Consumption   

                                    GOLD Description  \
0              The unique identifier for a customer.   
1             The year and month of the consumption.   
2  The consumption value for a customer in a spec...   

                                    LLM 1 Descripton Annotation LLM 1  \
0  Identifies a customer and is a foreign key tha...       4. Perfect   
1  Stores the date of consumption for each customer.       4. Perfect   
2  Records the consumption amount for a given cus...       4. Perfect   

                                    LLM 2 Descripton   Annotation LLM 2  \
0  The CustomerID column is used to uniquely iden...  3. Almost perfect   
1  The Date column in the yearmonth table represe...  3. Almost perfect   
2  The C

In [ ]:
import numpy as np
from sklearn.metrics import cohen_kappa_score

# Agreememt percentage
agreement = np.zeros((len(annotations), len(annotations)))
cohen_kappa = np.zeros((len(annotations), len(annotations)))

for anno_idx in range(len(annotations)):
    for i in range(len(annotations[anno_idx])):
        for anno_2_idx in range(len(annotations)):
            agreement[anno_idx, anno_2_idx] = np.mean(annotations[anno_idx]['Annotation'] == annotations[anno_2_idx]['Annotation'])
            cohen_kappa[anno_idx, anno_2_idx] = cohen_kappa_score(annotations[anno_idx]['Annotation'], annotations[anno_2_idx]['Annotation'])

agreement_df = pd.DataFrame(agreement, columns=annotator_names, index=annotator_names)
cohen_kappa_df = pd.DataFrame(cohen_kappa, columns=annotator_names, index=annotator_names)

print(f"The total number of annotations: {len(annotations[0])}\n")
print(f"The agreement percentage between the annotators:\n{agreement_df}\n")
print(f"The Cohen's Kappa between the annotators:\n{cohen_kappa_df}") 
# Kappa value interpretation Landis & Koch (1977):
# <0 No agreement
# 0 — .20 Slight
# .21 — .40 Fair
# .41 — .60 Moderate
# .61 — .80 Substantial
# .81–1.0 Perfect


The total number of annotations: 798

The agreement percentage between the annotators:
          Hjalmar     Sörme
Hjalmar  1.000000  0.864662
Sörme    0.864662  1.000000

The Cohen's Kappa between the annotators:
          Hjalmar     Sörme
Hjalmar  1.000000  0.713345
Sörme    0.713345  1.000000


In [28]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score

# Load human annotations
llm_judge_scores_path = "output/judge/judge_mistral7b.csv"
human_annotations_path = "annotations/LLM Prediction Annotations(13).xlsx"
human_annotations_df = pd.read_excel(human_annotations_path, sheet_name="Scores 123")
human_annotations_raw = human_annotations_df["Annotation 1 LLM 3"]

# Mapping of annotation values
annotation_mapping = {
    "4. Perfect": 4,
    "3. Almost perfect": 3,
    "2. Somewhat correct": 2,
    "1. Incorrect": 1
}


human_annotations = human_annotations_raw.map(annotation_mapping)

# Load LLM judge scores
llm_judge_scores_path = "output/judge/judge_qwen2-72B.csv"
llm_judge_scores_df = pd.read_csv(llm_judge_scores_path)
llm_judge_scores = llm_judge_scores_df["judgement"]

# Create a DataFrame combining both annotations and judge scores
combined_df = pd.DataFrame({
    "human_annotations": human_annotations,
    "llm_judge_scores": llm_judge_scores
})

# Drop rows with NaN values
combined_df = combined_df.dropna()


# Extract cleaned annotations and judge scores
human_annotations_clean = combined_df["human_annotations"].astype(int)
llm_judge_scores_clean = combined_df["llm_judge_scores"].astype(int)


# Ensure both series are of the same length
if len(human_annotations_clean) != len(llm_judge_scores_clean):
    raise ValueError("The lengths of cleaned human annotations and LLM judge scores do not match")

# Calculate Cohen's kappa
kappa = cohen_kappa_score(human_annotations_clean, llm_judge_scores_clean)

print(f"Cohen's kappa: {kappa}")


# # Apply the mapping to human annotations
# human_annotations = human_annotations_raw.map(annotation_mapping)

# # Load LLM judge scores

# llm_judge_scores_df = pd.read_csv(llm_judge_scores_path)
# llm_judge_scores = llm_judge_scores_df["judgement"]

# # Ensure both series are of the same length
# if len(human_annotations) != len(llm_judge_scores):
#     raise ValueError("The lengths of human annotations and LLM judge scores do not match")

# # Calculate Cohen's kappa
# kappa = cohen_kappa_score(human_annotations, llm_judge_scores)

# print(f"Cohen's kappa: {kappa}")


Cohen's kappa: 0.10952614357439028


In [2]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score

# Load the annotations
niklas = pd.read_excel('annotations/Difficulty Annotation.xlsx', sheet_name='Niklas')
ture = pd.read_excel('annotations/Difficulty Annotation.xlsx', sheet_name='Ture')


# def merge_categories(annotation):
#     if annotation in ['Medium', 'Hard']:
#         return 'Medium/Hard'
#     return annotation


# oscar['Merged Annotation'] = oscar['Annotation'].apply(merge_categories)
# ture['Merged Annotation'] = ture['Annotation'].apply(merge_categories)

# Calculate the Cohen's kappa coefficient
kappa = cohen_kappa_score(niklas['Annotation'], ture['Annotation'])

print(f"Cohen's kappa coefficient: {kappa}")

# kappa = cohen_kappa_score(oscar['Annotation'], ture['Annotation'])

# print(f"Merged Cohen's kappa coefficient: {kappa}")


Cohen's kappa coefficient: 0.6128300606320377


In [ ]:
0.31213204951856943